<a href="https://colab.research.google.com/github/obinnachike/Text-Summarizer/blob/main/text_summerizer_with_huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!invidia-smi

In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rough_score py7zr -q

In [ ]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

In [ ]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk, load_metric
import matplotlib.pyplot as plt
import pandas as pd
from transformers import AutoModelForSeq2seqLM, AutoTokenizer
import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import torch

nltk.download('punkt')

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrianed(model_ckpt)

In [ ]:
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

In [ ]:
dataset_samsum = load_dataset("samsum")

In [ ]:
dataset_samsum

In [ ]:
dataset_samsum['train']['dialogue'][1]

In [ ]:
dataset_samsum['train'][1]['summary']

In [ ]:
split_lengths = [len(dataset_samsum[split]) for split in dataset_samsum]

print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset_samsum['train'].column_names}")
print("\nDialogue")

print(dataset_samsum['test'][1]["dialogue"])

print("\nSummary")
print(dataset_samsum['test'][1]["summary"])

In [ ]:
def convert_examples_to_features(example_batch):
  input_encodings = tokenizer(example_batch['dialogue'], max_length = 1024, truncation = True)

  with tokenizer.as_target_tokenizer():
    target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True)

  return {
      'input_ids' : input_encodings['input_ids'],
      'attention_mask' : input_encodings['attention_mask']
      'labels' : target_encodings['input_ids']
  }

In [ ]:
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched = True)

In [ ]:
dataset_samsum_pt['train']

In [ ]:
dataset_samsum_pt['train']['input_ids'][1]

In [ ]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model = model_pegasus)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguements(
    output_dir = "pegasus-samsum",   #Output directory
    warmup_steps = 500,
    eval_strategy = "steps",   #Evaluate every epoch
    eval_steps = 500,
    save_steps = 1e6,
    learning_rate = 2e-5, # learning rate
    per_device_train_batch_size = 1, #Batch size for training
    per_device_eval_batch_size = 1, #Batch size for evaluation
    num_train_epochs = 1, #Number of training Epochs
    weight_decay = 0.01, # Strenght of weight decay
    logging_steps = 10,
    gradient_accumulation_steps = 16
)

In [ ]:
trainer = Trainer(
    model = model_pegasus,
    args = training_args,
    tokenizer = tokenizer,
    data_collator = seq2seq_data_collator,
    train_dataset = tokenized_datasets["train"],
    eval_dataset = tokenized-datasets["test"]
)

In [ ]:
trainer.train()